In [ ]:
from ultralytics import YOLO
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [ ]:
model = YOLO("yolo11n-cls.pt")

In [ ]:
model.train(
    data=r"C:\Users\dcmes\OneDrive\Desktop\Microproyecto_Cervical_Cancer_Classifier\dataset_preprocesado",
    epochs=100,
    imgsz=224,
    batch=128,
    device=0,
    #optimizer= "AdamW",
    lr0=0.05,
    #lrf= 0.1,
    #cos_lr=True,
    #mixup=0.1,
    #cutmix=0.1,
)

In [ ]:
# Ruta al conjunto de test
test_dir = r"C:\Users\dcmes\OneDrive\Desktop\Microproyecto_Cervical_Cancer_Classifier\dataset_preprocesado\test"
classes = ["class_0", "class_1"]

In [ ]:
y_true = []
y_scores = []

In [ ]:
# Recorremos ambas clases
for idx, cls in enumerate(classes):
    folder = os.path.join(test_dir, cls)
    for file in os.listdir(folder):
        if file.endswith((".png", ".jpg")):
            path = os.path.join(folder, file)
            results = model(path, verbose=False)

            # Probabilidades
            probs = results[0].probs.data.cpu().numpy()
            y_true.append(idx)        # etiqueta real
            y_scores.append(probs[1]) # prob de clase "1"

In [ ]:
y_true = np.array(y_true)
y_scores = np.array(y_scores)

roc_auc = roc_auc_score(y_true, y_scores)
print(f"AUC (roc_auc_score): {roc_auc:.4f}")

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_scores)
plt.figure()
plt.plot(fpr, tpr, color="blue", lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Cervical Cancer Classification")
plt.legend(loc="lower right")
plt.show()